In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 1927, done.
remote: Counting objects: 100% (1927/1927), done.
remote: Compressing objects: 100% (1684/1684), done.
remote: Total 1927 (delta 444), reused 769 (delta 225), pack-reused 0
Receiving objects: 100% (1927/1927), 30.09 MiB | 17.93 MiB/s, done.
Resolving deltas: 100% (444/444), done.


In [3]:
!pip install pycocotools

In [4]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1550866 sha256=8dc83b869c70b097a199354c048b414b9d489b968d7585d8a28a6f25fdb587f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-a82r0guf/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=50c248fcfded830ece345f1c8fdbeb602cc2a64c8ad8c28a3c10d2a366bb610d
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=cb425a8d22210e3491944ca6d8ed36f6075070333485a01ba3a3ddc0598ff4b4
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for oauth2client: filename=oauth2client-3.0.0-cp36-none-any.whl size=106382 sha256=d6af64da5abe8e943d492b58e7f02d7c4cd4ba226

ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: apache-beam 2.23.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [5]:
!git clone https://github.com/tarun-bisht/tensorflow-object-detection.git
os.chdir("tensorflow-object-detection")

Cloning into 'tensorflow-object-detection'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 32 (delta 5), reused 29 (delta 5), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [6]:
# Download a pretrained model from tensorflow model zoo
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
!rm efficientdet_d0_coco17_tpu-32.tar.gz
!mv efficientdet_d0_coco17_tpu-32 data/models

--2020-08-23 14:13:01--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.167.128, 2a00:1450:400c:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.167.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M  28.4MB/s    in 1.0s    

2020-08-23 14:13:03 (28.4 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



In [7]:
# load data from google drive for training
!unzip -q ../../gdrive/My\ Drive/detection_data_train.zip -d  ./data/images/train
!unzip -q ../../gdrive/My\ Drive/detection_data_test.zip -d  ./data/images/test

In [8]:
!python create_dataset.py --labels data/labels/theft.pbtxt

2020-08-23 14:15:32.880693: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully converted test annotations to csv at data/csv/test_labels.csv
Successfully created the TFRecords: data/tfrecords/test.record
Successfully converted train annotations to csv at data/csv/train_labels.csv
Successfully created the TFRecords: data/tfrecords/train.record


In [ ]:
!python train.py --model_dir= ../../gdrive/My\ Drive/models/efficientdet_d0_coco17_tpu-32-theft --pipeline_config_path=data/models/efficientdet_d0_coco17_tpu-32/pipeline.config

2020-08-23 14:21:57.321543: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 14:21:59.982947: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-23 14:22:00.007472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-23 14:22:00.008243: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-23 14:22:00.008292: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 14:22:00.010810: I tensorflow/stream_executor/platform/defa